**Dual LSTM Encoder for Dialog Response Generation**

http://www.wildml.com/2016/07/deep-learning-for-chatbots-2-retrieval-based-model-tensorflow/

https://github.com/dennybritz/chatbot-retrieval

https://github.com/rkadlec/ubuntu-ranking-dataset-creator

https://arxiv.org/abs/1506.08909

In [1]:
import tensorflow as tf
tf.VERSION

'1.2.0'

**Estimator**

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/estimator/Estimator

https://www.tensorflow.org/versions/r1.2/api_docs/python/tf/estimator

**model_fn**
```text
 |        model_fn: Model function. Follows the signature:
 |      
 |          * Args:
 |      
 |            * `features`: This is the first item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same.
 |            * `labels`: This is the second item returned from the `input_fn`
 |                   passed to `train`, 'evaluate`, and `predict`. This should be a
 |                   single `Tensor` or `dict` of same (for multi-head models). If
 |                   mode is `ModeKeys.PREDICT`, `labels=None` will be passed. If
 |                   the `model_fn`'s signature does not accept `mode`, the
 |                   `model_fn` must still be able to handle `labels=None`.
 |            * `mode`: Optional. Specifies if this training, evaluation or
 |                   prediction. See `ModeKeys`.
 |            * `params`: Optional `dict` of hyperparameters.  Will receive what
 |                   is passed to Estimator in `params` parameter. This allows
 |                   to configure Estimators from hyper parameter tuning.
 |            * `config`: Optional configuration object. Will receive what is passed
 |                   to Estimator in `config` parameter, or the default `config`.
 |                   Allows updating things in your model_fn based on configuration
 |                   such as `num_ps_replicas`, or `model_dir`.
 |      
 |          * Returns:
 |            `EstimatorSpec`
 ```

In [2]:
def dual_encoder(vocab_size,
                 embed_size,
                 hidden_size,
                 input_context,
                 input_context_len,
                 input_utterance,
                 input_utterance_len,
                 targets):

    input_data = tf.concat([input_context, input_utterance], axis=0)
    input_length = tf.concat([input_context_len, input_utterance_len], axis=0)
    input_length = tf.reshape(input_length, [-1])
    
    embeddings = tf.get_variable(
        'embeddings',
        shape=(vocab_size, embed_size),
        initializer=tf.random_uniform_initializer(-0.25, 0.25))

    input_embed = tf.nn.embedding_lookup(
        embeddings, input_data, name='input_embed')
        
    with tf.variable_scope('rnn'):
        cell = tf.nn.rnn_cell.LSTMCell(
            hidden_size,
            forget_bias=2.0,
            use_peepholes=True,
            state_is_tuple=True)

        outputs, states = tf.nn.dynamic_rnn(
            cell,
            input_embed,
            sequence_length=input_length,
            dtype=tf.float32)

        context_encoding, utterance_encoding = tf.split(
            states.h, num_or_size_splits=2, axis=0)

    with tf.variable_scope('prediction'):
        ct = context_encoding
        rt = utterance_encoding
        M = tf.get_variable(
            'M',
            shape=(hidden_size, hidden_size),
            initializer=tf.truncated_normal_initializer())

        ct_M = tf.matmul(ct, M)
        batch_ct_M = tf.expand_dims(ct_M, axis=2)
        batch_rt = tf.expand_dims(rt, axis=2)
        batch_ct_M_r = tf.matmul(batch_ct_M, batch_rt, transpose_a=True)
        ct_M_r = tf.squeeze(batch_ct_M_r, axis=2)

        b = tf.get_variable(
           'b', shape=(), initializer=tf.zeros_initializer())
        
        logits = ct_M_r + b
        
        probs = tf.sigmoid(logits)

    if targets is None:
        return probs, None

    loss = tf.losses.sigmoid_cross_entropy(
        multi_class_labels=targets, logits=logits, reduction=tf.losses.Reduction.MEAN)
    
    return probs, loss

In [3]:
def model_fn(features, labels, mode, params):
    vocab_size = params['vocab_size']
    embed_size = params['embed_size']
    hidden_size = params['hidden_size']

    input_context = features['context']
    input_context_len = features['context_len']
    input_utterance = features['utterance']
    input_utterance_len = features['utterance_len']

    probs, loss = dual_encoder(
        vocab_size,
        embed_size,
        hidden_size,
        input_context,
        input_context_len,
        input_utterance,
        input_utterance_len,
        labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        _learning_rate = params['learning_rate']
        _optimizer =  params['optimizer']
        
        train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=_learning_rate,
            clip_gradients=10.0,
            optimizer=_optimizer)
    else:
        train_op = None
    
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=probs,
        loss=loss,
        train_op=train_op)

**Input**

In [4]:
# `tokenizer` function must be defined before restoring the vocabulary object
# (pickle does not serialize functions)
def tokenizer(sentences):
    return (sentence.split() for sentence in sentences)

class VocabularyAdapter:
    
    def __init__(self, vocabulary_bin):
        self._vocab = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocabulary_bin)
    
    @property
    def size(self):
        return len(self._vocab.vocabulary_)
    
    @property
    def vector_length(self):
        return self._vocab.max_document_length


def input_features(vector_length):
    return [
        tf.feature_column.numeric_column(
            key='context', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='context_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance', shape=vector_length, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='utterance_len', shape=1, dtype=tf.int64),
        tf.feature_column.numeric_column(
            key='label', shape=1, dtype=tf.int64),
    ]


def input_fn(name, filenames, features, batch_size, num_epochs=None):
    example_features = tf.feature_column.make_parse_example_spec(features)

    batch_example = tf.contrib.learn.read_batch_record_features(
        file_pattern=filenames,
        features=example_features,
        batch_size=batch_size,
        num_epochs=num_epochs,
        randomize_input=True,
        queue_capacity=200000 + batch_size * 10,
        name='read_batch_record_features_' + name
    )

    batch_target = batch_example.pop('label')

    return batch_example, batch_target

**Training**

In [5]:
import os

HOME_DIR = 'ubuntu'
DATA_DIR = os.path.join(HOME_DIR, 'data')
VOCAB_BIN = os.path.join(DATA_DIR, 'vocabulary.bin')
TRAIN_TFR = os.path.join(DATA_DIR, 'train.tfrecords')
VALID_POSITIVE_TFR = os.path.join(DATA_DIR, 'valid_positive.tfrecords')
VALID_NEGATIVE_TFR = os.path.join(DATA_DIR, 'valid_negative.tfrecords')
TEST_POSITIVE_TFR = os.path.join(DATA_DIR, 'test_positive.tfrecords')
TEST_NEGATIVE_TFR = os.path.join(DATA_DIR, 'test_negative.tfrecords')

def has_file(file):
    if not os.path.isfile(file):
        raise Exception('File not found: {}'.format(file))

has_file(VOCAB_BIN)
has_file(TRAIN_TFR)
has_file(VALID_POSITIVE_TFR)
has_file(VALID_NEGATIVE_TFR)
has_file(TEST_POSITIVE_TFR)
has_file(TEST_NEGATIVE_TFR)

In [6]:
vocab = VocabularyAdapter(VOCAB_BIN)
features = input_features(vocab.vector_length)

In [7]:
params = {
    'vocab_size': vocab.size,
    'embed_size': 100,
    'hidden_size': 256,
    'learning_rate': 0.001,
    'optimizer': 'Adam',
    'batch_size': 256,
    'num_epochs': 2,
}

input_fn_train = lambda: input_fn('train', [TRAIN_TFR], features, params['batch_size'], 1)
input_fn_valid_positive = lambda: input_fn('valid_positive', [VALID_POSITIVE_TFR], features, 16, 1)
input_fn_valid_negative = lambda: input_fn('valid_negative', [VALID_NEGATIVE_TFR], features, 64, 1)
input_fn_valid = lambda: input_fn('valid', [VALID_POSITIVE_TFR, VALID_NEGATIVE_TFR], features, 64, 1)
input_fn_test = lambda: input_fn('test', [TEST_POSITIVE_TFR, TEST_NEGATIVE_TFR], features, 64, 1)

In [8]:
import shutil

def remove_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)

MODEL_DIR = os.path.join(HOME_DIR, 'model')

remove_dir(MODEL_DIR)

In [9]:
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    model_dir=MODEL_DIR,
    params=params)

estimator

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'ubuntu/model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


In [15]:
def epoch_range(i, num_epochs):
    range_start = i * num_epochs + 1
    range_end = range_start + num_epochs
    return range(range_start, range_end)

In [10]:
%%time

for epoch in epoch_range(0, params['num_epochs']):
    print('[ Epoch {} ]\n'.format(epoch))
    print('Training...\n')
    %time estimator.train(input_fn_train, steps=None)
    print()
    print('Validation...\n')
    %time estimator.evaluate(input_fn_valid_positive, steps=None, name='valid_positive')
    %time estimator.evaluate(input_fn_valid_negative, steps=None, name='valid_negative')
    print()

[ Epoch 1 ]

Training...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ubuntu/model/model.ckpt.
INFO:tensorflow:loss = 0.812547, step = 1
INFO:tensorflow:global_step/sec: 0.382038
INFO:tensorflow:loss = 0.677158, step = 101 (261.755 sec)
INFO:tensorflow:global_step/sec: 0.3822
INFO:tensorflow:loss = 0.693679, step = 201 (261.643 sec)
INFO:tensorflow:Saving checkpoints for 231 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.382055
INFO:tensorflow:loss = 0.679771, step = 301 (261.744 sec)
INFO:tensorflow:global_step/sec: 0.382513
INFO:tensorflow:loss = 0.652244, step = 401 (261.427 sec)
INFO:tensorflow:Saving checkpoints for 461 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.381934
INFO:tensorflow:loss = 0.663859, step = 501 (261.826 sec)
INFO:tensorflo

INFO:tensorflow:global_step/sec: 0.382046
INFO:tensorflow:loss = 0.467982, step = 4868 (261.749 sec)
INFO:tensorflow:global_step/sec: 0.382497
INFO:tensorflow:loss = 0.479199, step = 4968 (261.441 sec)
INFO:tensorflow:Saving checkpoints for 5048 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.382051
INFO:tensorflow:loss = 0.42306, step = 5068 (261.745 sec)
INFO:tensorflow:global_step/sec: 0.382374
INFO:tensorflow:loss = 0.441302, step = 5168 (261.523 sec)
INFO:tensorflow:global_step/sec: 0.38236
INFO:tensorflow:loss = 0.439145, step = 5268 (261.536 sec)
INFO:tensorflow:Saving checkpoints for 5278 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.381939
INFO:tensorflow:loss = 0.514743, step = 5368 (261.822 sec)
INFO:tensorflow:global_step/sec: 0.382313
INFO:tensorflow:loss = 0.434758, step = 5468 (261.563 sec)
INFO:tensorflow:Saving checkpoints for 5508 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.38191
INFO:tensorflow:loss = 0.464034,

In [12]:
%%time

for epoch in epoch_range(1, params['num_epochs']):
    print('[ Epoch {} ]\n'.format(epoch))
    print('Training...\n')
    %time estimator.train(input_fn_train, steps=None)
    print()
    print('Validation...\n')
    %time estimator.evaluate(input_fn_valid_positive, steps=None, name='valid_positive')
    %time estimator.evaluate(input_fn_valid_negative, steps=None, name='valid_negative')
    print()

[ Epoch 3 ]

Training...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-7334
INFO:tensorflow:Saving checkpoints for 7335 into ubuntu/model/model.ckpt.
INFO:tensorflow:loss = 0.342062, step = 7335
INFO:tensorflow:global_step/sec: 0.382352
INFO:tensorflow:loss = 0.402587, step = 7435 (261.542 sec)
INFO:tensorflow:global_step/sec: 0.382504
INFO:tensorflow:loss = 0.432555, step = 7535 (261.435 sec)
INFO:tensorflow:Saving checkpoints for 7565 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.381869
INFO:tensorflow:loss = 0.350354, step = 7635 (261.870 sec)
INFO:tensorflow:global_step/sec: 0.382304
INFO:tensorflow:loss = 0.381011, step = 7735 (261.573 sec)
INFO:tensorflow:Saving checkpoints for 7795 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/s

INFO:tensorflow:loss = 0.244352, step = 12102 (261.508 sec)
INFO:tensorflow:Saving checkpoints for 12151 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.382126
INFO:tensorflow:loss = 0.298475, step = 12202 (261.693 sec)
INFO:tensorflow:global_step/sec: 0.382643
INFO:tensorflow:loss = 0.343273, step = 12302 (261.340 sec)
INFO:tensorflow:Saving checkpoints for 12381 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.382233
INFO:tensorflow:loss = 0.248433, step = 12402 (261.620 sec)
INFO:tensorflow:global_step/sec: 0.382607
INFO:tensorflow:loss = 0.284591, step = 12502 (261.366 sec)
INFO:tensorflow:global_step/sec: 0.382382
INFO:tensorflow:loss = 0.289807, step = 12602 (261.521 sec)
INFO:tensorflow:Saving checkpoints for 12611 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.382
INFO:tensorflow:loss = 0.312805, step = 12702 (261.777 sec)
INFO:tensorflow:global_step/sec: 0.382474
INFO:tensorflow:loss = 0.251083, step = 12802 (261.456 sec)
INFO

In [16]:
%%time

for epoch in epoch_range(2, params['num_epochs']):
    print('[ Epoch {} ]\n'.format(epoch))
    print('Training...\n')
    %time estimator.train(input_fn_train, steps=None)
    print()
    print('Validation...\n')
    %time estimator.evaluate(input_fn_valid_positive, steps=None, name='valid_positive')
    %time estimator.evaluate(input_fn_valid_negative, steps=None, name='valid_negative')
    print()

[ Epoch 5 ]

Training...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-14668
INFO:tensorflow:Saving checkpoints for 14669 into ubuntu/model/model.ckpt.
INFO:tensorflow:loss = 0.224026, step = 14669
INFO:tensorflow:global_step/sec: 0.382176
INFO:tensorflow:loss = 0.241594, step = 14769 (261.662 sec)
INFO:tensorflow:global_step/sec: 0.382239
INFO:tensorflow:loss = 0.260846, step = 14869 (261.616 sec)
INFO:tensorflow:Saving checkpoints for 14899 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.381878
INFO:tensorflow:loss = 0.193899, step = 14969 (261.865 sec)
INFO:tensorflow:global_step/sec: 0.382276
INFO:tensorflow:loss = 0.192077, step = 15069 (261.589 sec)
INFO:tensorflow:Saving checkpoints for 15129 into ubuntu/model/model.ckpt.
INFO:tensorflow:glob

INFO:tensorflow:global_step/sec: 0.382276
INFO:tensorflow:loss = 0.126423, step = 19436 (261.592 sec)
INFO:tensorflow:Saving checkpoints for 19486 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.382
INFO:tensorflow:loss = 0.148968, step = 19536 (261.779 sec)
INFO:tensorflow:global_step/sec: 0.382456
INFO:tensorflow:loss = 0.187297, step = 19636 (261.468 sec)
INFO:tensorflow:Saving checkpoints for 19716 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.382099
INFO:tensorflow:loss = 0.147528, step = 19736 (261.712 sec)
INFO:tensorflow:global_step/sec: 0.382429
INFO:tensorflow:loss = 0.115976, step = 19836 (261.486 sec)
INFO:tensorflow:global_step/sec: 0.382493
INFO:tensorflow:loss = 0.179508, step = 19936 (261.443 sec)
INFO:tensorflow:Saving checkpoints for 19946 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.381873
INFO:tensorflow:loss = 0.169225, step = 20036 (261.867 sec)
INFO:tensorflow:global_step/sec: 0.382222
INFO:tensorflow:loss =

In [17]:
%%time

for epoch in epoch_range(3, params['num_epochs']):
    print('[ Epoch {} ]\n'.format(epoch))
    print('Training...\n')
    %time estimator.train(input_fn_train, steps=None)
    print()
    print('Validation...\n')
    %time estimator.evaluate(input_fn_valid_positive, steps=None, name='valid_positive')
    %time estimator.evaluate(input_fn_valid_negative, steps=None, name='valid_negative')
    print()

[ Epoch 7 ]

Training...

INFO:tensorflow:logits.dtype=<dtype: 'float32'>.
INFO:tensorflow:multi_class_labels.dtype=<dtype: 'float32'>.
INFO:tensorflow:losses.dtype=<dtype: 'float32'>.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ubuntu/model/model.ckpt-22002
INFO:tensorflow:Saving checkpoints for 22003 into ubuntu/model/model.ckpt.
INFO:tensorflow:loss = 0.117741, step = 22003
INFO:tensorflow:global_step/sec: 0.38275
INFO:tensorflow:loss = 0.108506, step = 22103 (261.269 sec)
INFO:tensorflow:global_step/sec: 0.382835
INFO:tensorflow:loss = 0.154423, step = 22203 (261.209 sec)
INFO:tensorflow:Saving checkpoints for 22233 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.38231
INFO:tensorflow:loss = 0.134092, step = 22303 (261.567 sec)
INFO:tensorflow:global_step/sec: 0.382618
INFO:tensorflow:loss = 0.127284, step = 22403 (261.357 sec)
INFO:tensorflow:Saving checkpoints for 22463 into ubuntu/model/model.ckpt.
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 0.382206
INFO:tensorflow:loss = 0.0503181, step = 26770 (261.640 sec)
INFO:tensorflow:Saving checkpoints for 26820 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.381848
INFO:tensorflow:loss = 0.104008, step = 26870 (261.886 sec)
INFO:tensorflow:global_step/sec: 0.382313
INFO:tensorflow:loss = 0.129855, step = 26970 (261.564 sec)
INFO:tensorflow:Saving checkpoints for 27050 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.381763
INFO:tensorflow:loss = 0.0994283, step = 27070 (261.943 sec)
INFO:tensorflow:global_step/sec: 0.38098
INFO:tensorflow:loss = 0.0839999, step = 27170 (262.480 sec)
INFO:tensorflow:global_step/sec: 0.382255
INFO:tensorflow:loss = 0.103239, step = 27270 (261.606 sec)
INFO:tensorflow:Saving checkpoints for 27279 into ubuntu/model/model.ckpt.
INFO:tensorflow:global_step/sec: 0.381798
INFO:tensorflow:loss = 0.0932593, step = 27370 (261.920 sec)
INFO:tensorflow:global_step/sec: 0.382237
INFO:tensorflow: